In [125]:
from langgraph.graph import START, END, StateGraph, MessagesState
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from CONFIG import GROQ_MODEL, TEMPERATURE
from langgraph.store.memory import InMemoryStore
from langgraph.store.base import BaseStore
from langchain_core.runnables import RunnableConfig
from langchain_core.messages import SystemMessage, HumanMessage

In [119]:
load_dotenv()
llm = ChatGroq(model=GROQ_MODEL, temperature=TEMPERATURE)

In [120]:
user_id = 'i1'
store = InMemoryStore()
user_details = ('user', user_id, 'detail')

In [121]:
store.put(user_details, 'personal_1', {'data': 'My name is Adnan saeed'})
store.put(user_details, 'personal_2', {'data': 'i am 24 year old and i did bachelor in artificial intelligence'})
store.put(user_details, 'professional_1', {'data': 'i am working on LLM based applications'})
store.put(user_details, 'professional_2', {'data': 'i did 3 months of internship at a startup as Python Intern'})
store.put(user_details, 'plans', {'data': 'my further plans move to Dubai for batter future, now i live in pakistan'})

In [122]:
# ----------------------------
# 2) System prompt template (your prompt)
# ----------------------------
SYSTEM_PROMPT_TEMPLATE = """You are a helpful assistant with memory capabilities.
If user-specific memory is available, use it to personalize 
your responses based on what you know about the user.

Your goal is to provide relevant, friendly, and tailored 
assistance that reflects the user’s preferences, context, and past interactions.

If the user’s name or relevant personal context is available, always personalize your responses by:
    – Always Address the user by name (e.g., "Sure, Nitish...") when appropriate
    – Referencing known projects, tools, or preferences (e.g., "your MCP  server python based project")
    – Adjusting the tone to feel friendly, natural, and directly aimed at the user

Avoid generic phrasing when personalization is possible. For example, instead of "In TypeScript apps..." 
say "Since your project is built with TypeScript..."

Use personalization especially in:
    – Greetings and transitions
    – Help or guidance tailored to tools and frameworks the user uses
    – Follow-up messages that continue from past context

Always ensure that personalization is based only on known user details and not assumed.

In the end suggest 3 relevant further questions based on the current response and user profile

The user’s memory (which may be empty) is provided as: {user_details_content}
"""

In [123]:
def chat(state: MessagesState, config: RunnableConfig, store: BaseStore):
    user_id = config['configurable']['user_id']
    user_detail = ('user', user_id, 'detail')
    items = store.search(user_detail)

    if items:
        paragraph = "\n".join(f"- {i.value.get('data', '')}" for i in items)

    else:
        paragraph = ''
        
    system_prompts = SYSTEM_PROMPT_TEMPLATE.format(
        user_details_content=paragraph
    )

    prompt = SystemMessage(content=system_prompts)

    resp = llm.invoke([prompt] + state['messages'])
    return {'messages': [resp]}   

In [128]:
config = {'configurable': {'user_id': 'i1'}}

builders = StateGraph(MessagesState)

builders.add_node('chat', chat)

builders.add_edge(START, 'chat')
builders.add_edge('chat', END)

graph = builders.compile(store=store)

result=graph.invoke(
    {
        'messages': [HumanMessage(content='what about UAE')]
    },
    config
)

print(result["messages"][-1].content)

Hi Adnan, since you're planning to move to Dubai for a better future, I'm sure you're aware that the UAE (United Arab Emirates) has a lot to offer, especially in the field of artificial intelligence and technology. With your background in AI and experience as a Python Intern, you might find opportunities to work on exciting LLM-based applications in the UAE.

The UAE has been investing heavily in AI research and development, and cities like Dubai and Abu Dhabi are becoming hubs for tech innovation. You might be interested to know that the UAE has launched several initiatives to promote AI adoption and development, such as the UAE AI Strategy 2031.

As someone who's worked on LLM-based applications, you might be excited to explore the potential for natural language processing and machine learning in the UAE's growing tech industry. With your plans to move to Dubai, you'll be well-positioned to take advantage of these opportunities and contribute to the UAE's thriving AI ecosystem.

Here